# Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, Dropout, Activation
from keras.preprocessing.image import ImageDataGenerator

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Import data

In [7]:
IMG_DIR = 'data/UTKFace'
IMG_SIZE = 224
CHANNELS = 3
NUM_CLASSES = 2
BATCH_SIZE = 32
NUM_EPOCHS = 50

data_aug_gen = ImageDataGenerator(rescale=1./255, horizontal_flip = True,
                                   validation_split = 0.2,
                                   preprocessing_function=tf.keras.applications.vgg19.preprocess_input)

train_gen = data_aug_gen.flow_from_directory(directory = IMG_DIR,
                                            subset = 'training',
                                            target_size = (IMG_SIZE, IMG_SIZE))

val_gen = data_aug_gen.flow_from_directory(directory = IMG_DIR, 
                                            subset = 'validation',
                                            target_size = (IMG_SIZE, IMG_SIZE))

Found 18965 images belonging to 2 classes.
Found 4740 images belonging to 2 classes.


# Model Definition

In [8]:
VGGFace = Sequential()

VGGFace.add(ZeroPadding2D(input_shape = (IMG_SIZE, IMG_SIZE, CHANNELS)))
VGGFace.add(Conv2D(64, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(64, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(128, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(128, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(256, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(256, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(256, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(ZeroPadding2D())
VGGFace.add(Conv2D(512, (3, 3), activation = 'relu'))
VGGFace.add(MaxPooling2D((2, 2), strides = 2))

VGGFace.add(Conv2D(4096, (7, 7), activation = 'relu'))
VGGFace.add(Dropout(0.5))

VGGFace.add(Conv2D(4096, (1, 1), activation = 'relu'))
VGGFace.add(Dropout(0.5))

VGGFace.add(Conv2D(2622, (1, 1)))
VGGFace.add(Flatten())
VGGFace.add(Activation('softmax'))

VGGFace.load_weights('data/vgg_face_weights.h5')

VGGFace.pop()
VGGFace.pop()
VGGFace.pop()

VGGFace.trainable = False

model = Sequential()

model.add(VGGFace)
model.add(Conv2D(NUM_CLASSES, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 1, 1, 4096)        134260544 
                                                                 
 conv2d_33 (Conv2D)          (None, 1, 1, 2)           8194      
                                                                 
 flatten_3 (Flatten)         (None, 2)                 0         
                                                                 
 activation_3 (Activation)   (None, 2)                 0         
                                                                 
Total params: 134,268,738
Trainable params: 8,194
Non-trainable params: 134,260,544
_________________________________________________________________


# Training

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', verbose = 1, patience = 5, mode = 'auto', restore_best_weights=True)

history = model.fit(train_gen, epochs = NUM_EPOCHS, validation_data = val_gen, callbacks = [earlystopping])

Epoch 1/50
271/593 [============>.................] - ETA: 41s - loss: 0.4976 - accuracy: 0.8564

KeyboardInterrupt: 

# Performance Evaluation

In [5]:
accuracy = history.history['accuracy']
loss = history.history['loss']
val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs = len(accuracy)
plt.figure(figsize=(15, 5))
plt.subplot(121)
plt.plot(accuracy[1:], label='Train_accuracy')
plt.plot(val_accuracy[1:], label='Val_accuracy')
plt.title('Accuracy over ' + str(epochs) + ' Epochs', size=15)
plt.legend()
plt.subplot(122)
plt.plot(loss[1:], label='Train_loss')
plt.plot(val_loss[1:], label='Val_loss')
plt.title('Loss over ' + str(epochs) + ' Epochs', size=15)
plt.legend()
plt.show()

# Prediction

In [7]:
# TODO: predict the submission data